<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies_noncovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6 MB 31.2 MB/s 
     |████████████████████████████████| 178 kB 59.0 MB/s 
     |████████████████████████████████| 3.0 MB 39.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires chardet<4,>=3.0.2, but you have chardet 4.0.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 8 30


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

Permission denied: https://drive.google.com/uc?id=1Sk56niKz9tu7M0LRBIZX0BP-PXp7ro8ubAY44s5XE6U
Maybe you need to change permission over 'Anyone with the link'?


In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210830 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210830 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [19]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [20]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [21]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [22]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v_nc > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v_nc > 0) & \
    (df_cap.icu_o_nc > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 
                              'icu_v_nc', 'nonicu_v_nc', 'mechvent_v_nc', 
                              'icu_o_nc', 'nonicu_o_nc', 'mechvent_o_nc',
                              'Landline Number', 'Landline Number 2', 'Fax Number', 
                              'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v_nc', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [23]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,30233,2021-08-27,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,87,2,5,23,7,4,(02) 8464 9999,None,(02) 8532 1764,Private
1,40378,2021-08-27,PHILIPPINE HEART CENTER,QUEZON CITY,39,179,48,23,137,18,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,39957,2021-08-27,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,29,37,5,6,62,2,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
3,5494,2021-08-27,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,26,190,25,36,91,19,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
4,148917,2021-08-27,PASAY CITY GENERAL HOSPITAL,PASAY CITY,26,50,8,6,23,1,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
5,71136,2021-08-27,MANILA DOCTORS HOSPITAL,ERMITA,26,47,10,6,96,4,(02) 8558 0888,None,(02) 8524 7376,Private
6,228497,2021-08-27,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,25,0,2,28,106,4,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
7,487999,2021-08-27,SAN LAZARO HOSPITAL,SANTA CRUZ,22,103,19,2,59,1,(02) 8309 9552,(02) 8711 6966,(02) 8711 6966,Government
8,25157,2021-08-27,"CAPITOL MEDICAL CENTER, INC.",QUEZON CITY,21,46,6,5,33,5,(02) 8372 3825,(02) 8938 7789,(02) 3411 4320,Private
9,441990,2021-08-27,POPE JOHN PAUL II HOSPITAL AND MEDICAL CENTER ...,CITY OF LAS PIÑAS,21,16,3,5,11,0,+63 88260285,None,None,Private


In [24]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,40379,2021-08-28,PHILIPPINE HEART CENTER,QUEZON CITY,40,196,51,24,120,15,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
1,5495,2021-08-28,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,31,187,25,31,84,13,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
2,39958,2021-08-28,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,29,51,5,6,48,2,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
3,71137,2021-08-28,MANILA DOCTORS HOSPITAL,ERMITA,28,51,9,4,92,5,(02) 8558 0888,None,(02) 8524 7376,Private
4,148918,2021-08-28,PASAY CITY GENERAL HOSPITAL,PASAY CITY,26,51,7,6,22,2,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
5,228498,2021-08-28,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,25,0,2,28,97,4,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
6,396925,2021-08-28,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,22,36,6,11,34,2,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
7,488000,2021-08-28,SAN LAZARO HOSPITAL,SANTA CRUZ,21,89,5,3,73,8,(02) 8309 9552,(02) 8711 6966,(02) 8711 6966,Government
8,441991,2021-08-28,POPE JOHN PAUL II HOSPITAL AND MEDICAL CENTER ...,CITY OF LAS PIÑAS,21,16,3,4,13,0,+63 88260285,None,None,Private
9,155696,2021-08-28,MCU-FDT MEDICAL FOUNDATION HOSPITAL,CALOOCAN CITY,21,35,13,4,23,2,(02) 8367 2031,(02) 8365 4868,(02) 8361 4664,Private


In [25]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,30235,2021-08-29,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,85,1,5,25,8,3,(02) 8464 9999,None,(02) 8532 1764,Private
1,40380,2021-08-29,PHILIPPINE HEART CENTER,QUEZON CITY,40,196,51,24,120,15,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,5496,2021-08-29,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,32,185,25,30,96,18,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,39959,2021-08-29,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,27,50,4,8,49,3,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
4,71138,2021-08-29,MANILA DOCTORS HOSPITAL,ERMITA,27,67,9,5,76,5,(02) 8558 0888,None,(02) 8524 7376,Private
5,228499,2021-08-29,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,27,0,3,26,110,3,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
6,148919,2021-08-29,PASAY CITY GENERAL HOSPITAL,PASAY CITY,26,53,8,6,20,1,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
7,396926,2021-08-29,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,22,22,6,11,48,2,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
8,488001,2021-08-29,SAN LAZARO HOSPITAL,SANTA CRUZ,21,105,2,3,57,2,(02) 8309 9552,(02) 8711 6966,(02) 8711 6966,Government
9,441992,2021-08-29,POPE JOHN PAUL II HOSPITAL AND MEDICAL CENTER ...,CITY OF LAS PIÑAS,21,16,3,5,16,0,+63 88260285,None,None,Private


In [26]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o_nc'].sum()
vacant = df_cap[filter]['icu_v_nc'].sum()
pct_used = (occupied / (vacant + occupied)) * 100